In [179]:
from math import ceil
import time, re, ast, sys, urllib, random, string, requests
import pandas as pd
from bs4 import BeautifulSoup
from random import randint
from selenium.webdriver.firefox.webdriver import FirefoxProfile
from urllib.request import urlopen, Request
from urllib.error import HTTPError, URLError
from itertools import cycle
from header_list import user_agent_list
from proxies_list import proxies_list_
from INITIAL_SCRAPPER_FUNCTIONS import *
from PARSER_FUNCTIONS import *
from LIST_DF_FUNCTIONS import *
from SEARCH_URL_GEN import *
from GET_SEARCH_URL import *
ua = user_agent_list
proxies = proxies_list_

In [152]:
customer_url = input("Input the Redfin URL of the Home: ")

Input the Redfin URL of the Home:  https://www.redfin.com/WA/Seattle/619-NW-75th-St-98117/home/495668


In [154]:
header = random.sample(ua, 1)[0]
proxy = random.sample(proxies, 1)[0]
url = customer_url
print(proxy)
print(url)
soup = session_creator(proxy, ua, url)
customer_df = info_from_property(soup)

24.227.222.4:53281
https://www.redfin.com/WA/Seattle/619-NW-75th-St-98117/home/495668


In [180]:
def gen_url(customer_df):
    zip_code = int(customer_df['zip_code'][1])
    city = customer_df['city'][1].replace(',', '')
    type_home = customer_df['style'][1]
    price = float(customer_df['last_sold_price'][
                  1].replace('$', '').replace(',', ''))
    num_bds = int(customer_df['num_bdrms'][1])
    num_bths = float(customer_df['num_bths'][1])
    sqft = int(customer_df['sqft'][1].replace(',', '').replace(' ', ''))
    yr_blt = int(customer_df['yr_blt'][1])
    lot_sqft = customer_df['lot_size'][1].replace(',', '').replace(' ', '').replace('—', '0')
    
    if ('Sq' in lot_sqft or 'Ft' in lot_sqft):
        lot_sqft = int(''.join(num for num in re.findall(r'\d', lot_sqft)))
    elif ('ac' in lot_sqft or 're' in lot_sqft):
        lot_sqft = int(float(''.join(num for num in re.findall(r'\d?\.?\d?\d?', lot_sqft)))*43560)
    elif (float(lot_sqft)>0 and float (lot_sqft)<10):
        lot_sqft = int(float(''.join(num for num in re.findall(r'\d?\.?\d?\d?', lot_sqft)))*43560)
    else:
        lot_sqft = int(''.join(num for num in re.findall(r'\d', lot_sqft)))

    if 'hoa' in customer_df.columns:
        hoa = customer_df['hoa'].values
        hoa_fee = float(customer_df['hoa_fee'].values)
        url_part_nine = ',' + search_url_part_nine_gen(hoa_fee)
    else:
        url_part_nine = ''

    url_part_one = 'https://www.redfin.com/zipcode/' + \
        str(zip_code) + '/filter/sort=lo-days'

    url_part_two = ',' + search_url_part_two_gen(type_home)
    if len(url_part_two) <= 1:
        url_part_two = ''

    url_part_three = ',' + search_url_part_three_gen(price)
    if len(url_part_three) <= 1:
        url_part_three = ''

    url_part_four = ',' + search_url_part_four_gen(num_bds)
    if len(url_part_four) <= 1:
        url_part_four = ''

    url_part_five = ',' + search_url_part_five_gen(num_bths)
    if len(url_part_five) <= 1:
        url_part_five = ''

    url_part_six = ',' + search_url_part_six_gen(sqft)
    if len(url_part_six) <= 1:
        url_part_six = ''

    url_part_seven = ',' + search_url_part_seven_gen(yr_blt)
    if len(url_part_seven) <= 1:
        url_part_seven = ''
    
    url_part_eight = ',' + search_url_part_eight_gen(lot_sqft)
    if (len(url_part_eight) <= 1 or (type_home == 'Condo/Co-op' or type_home == 'Townhouse')):
        url_part_eight = ''

    search_url = url_part_one + url_part_two + url_part_three + \
        url_part_four + url_part_five + url_part_six + \
        url_part_seven + url_part_eight + url_part_nine

    return search_url

In [181]:
gen_url(customer_df)

'https://www.redfin.com/zipcode/98117/filter/sort=lo-days,min-price=750k,max-price=900k,min-beds=3,max-beds=4,min-baths=1,min-sqft=1.25k-sqft,max-sqft=1.75k-sqft,min-year-built=1940,max-year-built=1960,min-lot-size=2k-sqft,max-lot-size=6.5k-sqft'